In [1]:
import numpy as np
import pandas as pd
import pandas_datareader
from types import SimpleNamespace
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

# Import additional libraries: 

# 1. Regional CO$_2$ emissions

In this exercise, we will investigate the development in CO2 emissions relative to GDP across the world. For this purpose, we apply data obtained from the World Bank Indicators.   

You will need to use the following data sets for the period 1990-2016:
1. **EN.ATM.CO2E.KT** : CO2 emissions in kilotons. [WB link](https://data.worldbank.org/indicator/EN.ATM.CO2E.KT) 
2. **NY.GDP.MKTP.PP.KD** : GDP, PPP-corrected in constant 2017 international dollars [WB link](https://data.worldbank.org/indicator/NY.GDP.MKTP.PP.KD)
3. **SP.POP.TOTL** : total population [WB link](https://data.worldbank.org/indicator/SP.POP.TOTL)
4. **country_region.xlsx** : key between country names and world regions

Data sets 1-3 contain annual records for all the world's countries plus a large set of country aggregates. You can download each `data_set` using the following API call

In [2]:
from pandas_datareader import wb
df = wb.download(indicator=data_set, country=[], start=1990, end=2016)

**Notes:**
The data set **country_region.xlsx** is available together with this notebook.   
Setting `countrty=[]` implies getting all countries in a data set by the syntax of the World Bank API.   
Follow the links above if there is a problem with the API call and download manually.    

**Question 1:** Data cleaning. 
* Download data sets 1-3. Exclude all entries in the data sets that are not a country listed in country_region.xlsx. 
* Assign a world region to each country in data sets 1-3 using country_region.xlsx. 
* If a data-point in one data set is `NaN` for country $x$ in year $y$, then delete it together with the corresponding data-points for country $x$ in year $y$ in the 2 other data sets.

Check that the resulting number of rows for a dataset containing GDP, population and CO2 is 4660. 

**Question 2:** Regional CO2 emissions. Calculate the regional CO2 emissions per dollar GDP on an annual basis. That is, for region $R$ in year $t$ compute
$$
\frac{\sum_{j \in R} \text{CO2}_{j,t}}{\sum_{j \in R} Y_{j,t}}
$$
where $Y_{j}$ denotes the GDP of country $j$. 

Plot CO2 emissions per dollar GDP for all regions in the way you find most presentable and comment. 

**Question 3:** Growth rates. 
* For each region $R$, calculate population weighted average $\text{CO2}_{R,t}$  and GDP pr year. See weighted average definition below.
* Calculate the annual **growth rates** of averaged GDP and CO2 for each region.
* Finally, create **one subplot** per region containing the two associated growth rates. Make a brief comment. (*tip: better use a loop for this instead of code repetition*)

The weighted averages of of GDP and CO2 for region $R$ is obtained by first calculating the weights:
$$
w_{i,t} = \frac{POP_{i,t}}{\sum \limits_{k \in R}POP_{k,t}}
$$
for each country $i \in R$.   
Then get average regional GDP by
$$
\bar{Y}_{R,t} =  \sum \limits_{i \in R}w_{i,t}Y_{i,t}
$$
and similarly for CO2.

The growth rate for regional GDP per capita is then 
$$
\bar{Y}^{\Delta}_{R,t} = \frac{\bar{Y}_{R,t}}{\bar{Y}_{R,t-1}} - 1
$$
and similarly for regional CO2 pr capita

# 2. Risky assets

The consumption savings model with uncertainty in both income and interest rate.

A **household** lives two periods.

In the **second period** it gets utility from **consuming** and **leaving a bequest**,

$$
\begin{aligned}
v_{2}(m_{2})&= \max_{c_{2}}\frac{c_{2}^{1-\rho}}{1-\rho}+\nu\frac{(a_2+\kappa)^{1-\rho}}{1-\rho}\\
\text{s.t.} \\
a_2 &= m_2-c_2 \\
a_2 &\geq 0
\end{aligned}
$$

* $m_2$ is cash-on-hand 
* $c_2$ is consumption
* $a_2$ is end-of-period assets 
* $\rho > 1$ is the risk aversion coefficient
* $\nu > 0 $ is the strength of the bequest motive
* $\kappa > 0$ is the degree of luxuriousness in the bequest motive  
* $a_2\geq0$ ensures the household *cannot* die in debt

The **value function** $v_t(m_t)$ measures the household's value of having $m_t$ at the beginning of period $t$.

In the **first period**, the household gets utility from consuming and takes into account that it will also live in the next-period, where it receives a stochastic income.   

We assume that the consumption decision is made at the **beginning** of the period.   

We denote the interest rate on savings from period 1 to 2 by $r_{1,2}$. It is assumed to be **unknown** until the end of the period 1; ie. *after* the consumption decision is made.  

$$
\begin{aligned}
	v_1(m_1) &= \max_{c_1}\frac{c_{1}^{1-\rho}}{1-\rho}+\beta\mathbb{E}_{1}\left[v_2(m_2)\right]\\&\text{s.t.}&\\
	a_1 & = m_1-c_1\\
	m_2 &= (1+r_{1,2})a_1+y_2 \\
	r_{1,2} &= \begin{cases}
	r_0 - \Delta^r & \text{with prob. }\frac{1}{2}\\
	r_0 + \Delta^r & \text{with prob. }\frac{1}{2} \\
	\end{cases}\\
	y_{2} &= \begin{cases}
	1-\Delta^y & \text{with prob. }\frac{1}{2}\\
	1+\Delta^y & \text{with prob. }\frac{1}{2} 
	\end{cases}\\
	a_1 & \geq0
\end{aligned}
$$

* $m_1$ is cash-on-hand in period 1
* $c_1$ is consumption in period 1
* $a_1$ is end-of-period assets in period 1
* $\beta > 0$ is the discount factor
* $\mathbb{E}_1$ is the expectation operator conditional on information in the *beginning* of period 1
* $y_2$ is income in period 2
* $\Delta^y \in (0,1)$ is the level of income risk
* $r_0$ is the expected interest rate
* $\Delta^r \in (0,r_0)$ is the level of interest rate risk
* $a_1\geq0$ ensures the household *cannot* borrow

In [1]:
# Parameters
rho = 8
kappa = 0.5
nu = 0.2
r0 = 0.3
Delta_r = 0.29
Delta_y = 0.5
beta = 0.98

Note: $r_0$ and $\Delta^r$ are somewhat extreme - that is just for exposition. 

**Question 1:** Solve the model for both periods and obtain value functions $v_1(m_1), v_2(m_2)$ together with the optimal consumption functions $c^*_1(m_1), c^*_2(m_2)$.   
Plot $v_1(m_1), v_2(m_2)$ in one graph and  $c^*_1(m_1), c^*_2(m_2)$ in another. 

**Question 2:** Now set $\Delta^r = 0$ and solve the model once again. Plot the associated $c^*_1(m_1), c^*_2(m_2)$ and compare them with the consumption functions from the solution in Question 1.

**Question 3** Question 3 below has **2 options**. You can **freely choose** which one you want to answer. (*You can of course answer both if you like*). 

**Question 3 - option 1:** Simulate the period 2 choices of $N$ households both for the case where $\Delta^r=0.29$ and where $\Delta^r=0$.   
You can use the same distribution of $m_1$ for both cases as specified below.   
Plot the two distributions of $c_1$ and comment. 
Optionally, you can also inspect the two distributions of $c_2$ and check that you understand their shapes.  

In [ ]:
np.random.seed(2021)
simN = 50000
sim_m1 = np.fmax(np.random.normal(1, 0.05, size = simN), 0)

**Question 3 - option 2:** Generalizing the set of possible interest rate outcomes.   
We now consider the case where the interest rate has $N = 2n$ different possible realizations.   
Specifically, $r$ has $n$ possible outcomes higher than $r_0$, and $n$ outcomes lower than $r_0$, which are **uniformly** distributed:
$$
\begin{aligned}
	r &= \begin{cases}
	r_0 - n\Delta^r & \text{with prob. }\frac{1}{N}\\
	r_0 - (n-1)\Delta^r & \text{with prob. }\frac{1}{N}\\
	r_0 - (n-2)\Delta^r & \text{with prob. }\frac{1}{N}\\
    \vdots & \\
	r_0 - \Delta^r & \text{with prob. }\frac{1}{N}\\
	r_0 + \Delta^r & \text{with prob. }\frac{1}{N}\\
    \vdots & \\
	r_0 + (n-1)\Delta^r & \text{with prob. }\frac{1}{N} \\
	r_0 + n\Delta^r & \text{with prob. }\frac{1}{N} \\
	\end{cases}\\
\end{aligned}
$$

Implement this generalized specification in code.   
Test the model on the parameterization below and plot $c^*_1(m_1), c^*_2(m_2)$. 

In [11]:
# New parameters:
r0 = 0.3
Delta_r = 0.05
n = 5
N = 2*5
# Remaining parameters are the same as in Question 1

# 3. Interpolation by polynomial

We can use interpolation when dealing with some function $f$ that is very heavy to  calculate. That is, we have a large set of points $X = \{x_i\}_{i=1}^{N}$ on which we want to know the function value $f(x_i)$, but we might not have sufficiently computing power for it.   

Therefore, we take out a subset $Z \subset X$ and calulate $f$ on the elements of $Z$. Whenever we want to know $f(x)$ for some $x\in X$ that is not also in our pre-computation set $Z$, we use interpolation between neighboring points to $x$ in $Z$ for which the function value has been calculated. To get an **estimate** of $f(x)$, we then interpolate between these pre-calculated neighboring points. 

In lecture 11, we saw how to write up a *linear interpolation* between a set of pre-calculated function values. 

In the current exercise, we will use **a polynomial** to interpolate between pre-calculated data points. 

(*Note: the type of interpolation we consider below is only practical in a modified version. It is useful as an introduction, however.*) 

**Polynomial interpolation**   
Assume we have a set of data points $D = \{(y_i,x_i)\}_{i=1}^{n}$.  

We now want to create a polynomial $p$ of degree $n-1$ such that $y_i = p(x_i)$ *exactly* for all $x_i,y_i \in D$.

We can obtain such a polynomial $p$ by the following steps:
$$
\begin{align}
	p(x) & = \sum \limits_{i = 1}^{n} y_i \times l_i(x) \\
	l_i(x) & = \prod \limits_{j \ne i} \frac{x - x_j}{x_i - x_j}
\end{align}
$$

**Example**   
Say we have the function $f(x) = x^2 - 1$. We can then construct our set of data points $D$.   
To do so, we evaluate $f$ in the points $\{1, 3, 5\}$:  
* $f(1) = 0$
* $f(3) = 8$
* $f(5) = 24$

Thus we have $D = \{(0,1), (8,3), (24,5)\}$

Interpolating our function $f(x) = x^2 - 1$ at some $\hat{x} \in [1,5]$ is then given by

$$
p(\hat{x})  =  0 \times l_1(\hat{x}) + 3 \times l_2(\hat{x}) + 5 \times l_3 (\hat{x}) 
$$


$$
\begin{align}
	l_1(\hat{x}) = \frac{(\hat{x} - 3)}{(1 - 3)} \frac{(\hat{x} - 5)}{(1 - 5)}, \:\: 
    l_2(\hat{x}) = \frac{(\hat{x} - 1)}{(3 - 1)} \frac{(\hat{x} - 5)}{(3 - 5)}, \:\: 
    l_3(\hat{x}) = \frac{(\hat{x} - 1)}{(5 - 1)} \frac{(\hat{x} - 3)}{(5 - 3)}
\end{align}
$$

**Algorithm**   
First evaluate the function $f$ on the set of points $X$ so as to obtain the associated function values $Y$. Then follow the pseudo-code below.     

<img src="poly_algo_fig.png" style="width: 300px;"/> 

**Question 1:** Implement the algorithm to interpolate function values of $f$ to the point $\hat{x}$ 

In [15]:
def polynomial_interpol(x, X, Y):
    pass

  You can test your `polynomial_interpol` on the function $f(x) = x^3$ 

In [ ]:
f = lambda x: x**3

X1 = np.array([1, 2, 2.5, 4])
X2 = np.array([0.5, 2.2, 7, 12.9])

Y1 = f(X1)
Y2 = f(X2)

x = 3.2 

# Note: the result should be the same in both cases below 
y1_interpol = polynomial_interpol(x, X1, Y1)
print(f'f(x) = {f(x): .7f}  interpolated y = {y1_interpol: .7f}')

y2_interpol = polynomial_interpol(x, X2, Y2)
print(f'f(x) = {f(x): .7f}  interpolated y = {y2_interpol: .7f}')
